<a href="https://colab.research.google.com/github/sortira/aiml-works/blob/main/NerdFish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nerdfish

A tool for nerds to catfish using image filters, AI or non-AI

In [1]:
import random, cv2, torch
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np
from transformers import *
from diffusers import StableDiffusionImg2ImgPipeline

GroupViT models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version.Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.
TAPAS models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version. Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.


In [10]:
def noise(image, noise_factor=0.2):
    np_image = np.array(image)
    noise = np.random.normal(scale=noise_factor * 255, size=np_image.shape)
    noisy_image = np_image + noise
    noisy_image = np.clip(noisy_image, 0, 255)
    return Image.fromarray(noisy_image.astype(np.uint8))

In [11]:
def vintage(image):
    width, height = image.size
    pixels = image.load()
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            tr = int(0.393 * r + 0.769 * g + 0.189 * b)
            tg = int(0.349 * r + 0.686 * g + 0.168 * b)
            tb = int(0.272 * r + 0.534 * g + 0.131 * b)
            r, g, b = min(tr, 255), min(tg, 255), min(tb, 255)
            pixels[x, y] = (r, g, b)
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.2)
    return image

In [12]:
def vignette(image):
    width, height = image.size
    cx, cy = width / 2, height / 2
    strength = 0.5
    vig_img = image.convert('RGBA')
    pixels = vig_img.load()
    for y in range(height):
        for x in range(width):
            d = ((x - cx) / cx) ** 2 + ((y - cy) / cy) ** 2
            pixels[x, y] = tuple(int(v * (1 - strength * d)) for v in pixels[x, y])
    return vig_img.convert('RGB')

In [13]:
def scanlines(image, line_intensity=80, line_spacing=5):
    width, height = image.size
    pixels = image.load()

    for y in range(0, height, line_spacing):
        for x in range(width):
            r, g, b = pixels[x, y]
            pixels[x, y] = (max(r - line_intensity, 0), max(g - line_intensity, 0), max(b - line_intensity, 0))

    return image

In [14]:
def colordistort(image, strength=20):
    width, height = image.size
    pixels = image.load()

    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            r = max(0, min(255, r + random.randint(-strength, strength)))
            g = max(0, min(255, g + random.randint(-strength, strength)))
            b = max(0, min(255, b + random.randint(-strength, strength)))
            pixels[x, y] = (r, g, b)

    return image

In [15]:
def pixelate(image, pixel_size=9):
    width, height = image.size
    image = image.resize(
        (width // pixel_size, height // pixel_size),
        resample=Image.NEAREST
    )
    image = image.resize(
        (width, height),
        resample=Image.NEAREST
    )
    return image

In [16]:
def scratch(image, scratch_factor=0.08):
    np_image = np.array(image)
    height, width, _ = np_image.shape

    for _ in range(int(scratch_factor * width * height)):
        x = np.random.randint(0, width)
        y = np.random.randint(0, height)
        np_image[y, x] = [255, 255, 255]
    return Image.fromarray(np_image)

In [17]:
def glitch(image, glitch_intensity=50, shift_max=15):
    np_image = np.array(image)
    height, width, _ = np_image.shape
    for _ in range(glitch_intensity):
        row = random.randint(0, height - 1)
        shift = random.randint(-shift_max, shift_max)
        np_image[row] = np.roll(np_image[row], shift, axis=0)
        if random.random() < 0.5:
            np_image[row] = np_image[row] + np.random.randint(-50, 50, np_image[row].shape)

    np_image = np.clip(np_image, 0, 255)
    return Image.fromarray(np_image.astype(np.uint8))

In [18]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--openai--clip-vit-base-patch16/snapshots/57c216476eefef5ab752ec549e440a49ae4ae5f3/config.json
`text_config` is `None`. Initializing the `CLIPTextConfig` with default values.
`vision_config` is `None`. initializing the `CLIPVisionConfig` with default values.
Model config CLIPConfig {
  "architectures": [
    "CLIPModel"
  ],
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "clip",
  "projection_dim": 512,
  "text_config": {
    "bos_token_id": 0,
    "dropout": 0.0,
    "eos_token_id": 2,
    "model_type": "clip_text_model"
  },
  "torch_dtype": "float32",
  "transformers_version": "4.47.1",
  "vision_config": {
    "dropout": 0.0,
    "model_type": "clip_vision_model",
    "patch_size": 16
  }
}

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so t

In [2]:
def filter_based_on_prompt(image_path, prompt, strength=0.75, guidance_scale = 7.5):
  pipe = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16,)
  pipe = pipe.to("cuda")
  init_image = Image.open(image_path).convert("RGB")
  init_image = init_image.resize((512, 512))
  modified_image = pipe(prompt=prompt, image=init_image, strength=strength, guidance_scale=guidance_scale).images[0]

  return modified_image

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

loading configuration file /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14/feature_extractor/preprocessor_config.json
size should be a dictionary on of the following set of keys: ({'height', 'width'}, {'shortest_edge'}, {'longest_edge', 'shortest_edge'}, {'longest_edge'}, {'max_height', 'max_width'}), got 224. Converted to {'shortest_edge': 224}.
crop_size should be a dictionary on of the following set of keys: ({'height', 'width'}, {'shortest_edge'}, {'longest_edge', 'shortest_edge'}, {'longest_edge'}, {'max_height', 'max_width'}), got 224. Converted to {'height': 224, 'width': 224}.
Image processor CLIPImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "CLIPImageProcessor",
  "im

  0%|          | 0/37 [00:00<?, ?it/s]